# Author Prediction
- checking gpu stats and limiting gpu memory
- importing dataset from pickle

In [30]:
from platform import python_version
# import ipyparallel as ipp
# c = ipp.Client(profile="slurm")
# c.ids

print(python_version())

3.10.4


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle
from keras import backend as K
# import pickle5 as pickle

is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=2024)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
print("cuda gpu is available: {}".format(is_cuda_gpu_available))

#file_name = "data/vision_forward_graph_data_local_05_08_22.pkl"
file_name = "data/vision_forward_graph_data_08_09_22.pkl"
with open(file_name, 'rb') as f:
    df = pickle.load(f)

df.shape

cuda gpu is available: False


(3419848, 127)

In [32]:
# importing utility functions
%run author_vision_util.ipynb

In [33]:
df = equalize_samples(df)
df = df[df["platform"] == "reddit"]
df.shape

/local/jobs/12816994/ipykernel_26085/2857450278.py:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  chosen_conversation_ids = sample(df_conversations, n)


chosen 507 conversations and gotten 517790 from twitter compared to 514793 from reddit


(514793, 127)

#### Create a one hot vector representation of the possible authors
- create an artificial user that represents a new user in a conversation up to that point
- get a matrix with the authors as columns and a 1 if the author wrote the post
- join it with the feature matrix
- drop the author column


In [34]:
# compute a fake user that symbolizes that the given user has not been seen at a given stage in the conversation
df_conversation_authors = df[["conversation_id", "author", "current_time"]]
first_times = df_conversation_authors.groupby(["conversation_id", "author"]).min()

def is_new_author(row):
    earliest_author_post = first_times.loc[row["conversation_id"],row["author"]]
    current_post_time = row["current_time"]
    return  earliest_author_post >= current_post_time

new_author_column = df[["conversation_id", "author", "current_time"]].apply(is_new_author, axis=1)
new_author_column= new_author_column.rename(columns={'current_time':"Author_is_new"})
#new_author_column.describe()
# current author has not been the beam_node
new_author_column.value_counts()

Author_is_new
False            260759
True             254034
dtype: int64

In [35]:
def compute_new_author_column(df):
    author_one_hot = pd.get_dummies(df.author, prefix="Author", sparse=True)
    # make author cells 0 that are now represented as "new author"
    author_one_hot = author_one_hot.astype(bool).apply(lambda x: x & ~new_author_column.Author_is_new).astype(int)
    # delete columns that are all 0 
    author_one_hot = author_one_hot.loc[:, (author_one_hot != 0).any(axis=0)]
    # join the new author column to the labels
    labels = author_one_hot.join(new_author_column.astype(int))
    features = take_features(df, ["author", "current_time", "beam_node_time"])
    combined_set = features.join(labels)
    return combined_set, features, labels

combined_set, features, labels = compute_new_author_column(df)
# combined_set.head()



In [36]:
# combined_set = combined_set.sparse.to_coo()

#### Training NN to predict the author that would write next
- included a "new author" category to capture predicting unknown authors
- using multi-class classification (instead of multi-label)
- relu/sigmoid activation functions have same effect
- precision grew significantly when adding more than 3-5 layers

In [37]:
from keras.layers import Dropout
from keras.optimizer_v2.rmsprop import RMSprop  # selecting train and test datasets
train, test = train_test_split(combined_set, test_size=0.2, shuffle=False)
print("split training and test set")

# Shuffle your dataset 
# shuffle_df = combined_set.sample(frac=1)

# Define a size for your train set 
# train_size = int(0.7 * len(combined_set))

# Split your dataset 
# train = shuffle_df[:train_size]
# test = shuffle_df[train_size:]


split training and test set


In [38]:
# train the model
y = train.drop(features.columns, axis=1)
x = train.drop(labels.columns, axis=1)
print("seperated features and y with shapes:")
print(x.shape)
print(y.shape)

# import tensorflow and train the model
# print(tf.__version__)
input_shape = (x.shape[1],)
output_shape = y.shape[1]
print("inputshape is {}".format(input_shape))
model = Sequential([
    Dense(output_shape, activation='relu', input_shape=input_shape),
    Dense(output_shape, activation='relu', input_shape=input_shape),
    Dense(output_shape, activation='relu', input_shape=input_shape),        
    Dense(output_shape, activation='relu', input_shape=input_shape),
    Dense(output_shape, activation='relu', input_shape=input_shape),
    Dense(output_shape, activation='relu', input_shape=input_shape),    
    Dense(output_shape, activation='softmax', input_shape=input_shape)
])
print("defined model as {}".format(model.layers))
# stochastic gradient descend as a classifier seem appropriate
model.compile(
    optimizer=RMSprop(),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy', 'accuracy' ,'mae']
)
print("compiled model")

seperated features and y with shapes:
(411834, 117)
(411834, 3653)
inputshape is (117,)
defined model as [<keras.layers.core.dense.Dense object at 0x2aab608a2b30>, <keras.layers.core.dense.Dense object at 0x2aab52460b80>, <keras.layers.core.dense.Dense object at 0x2aab524cbb20>, <keras.layers.core.dense.Dense object at 0x2aaaae81faf0>, <keras.layers.core.dense.Dense object at 0x2aaaae81e2f0>, <keras.layers.core.dense.Dense object at 0x2aaaae81da80>, <keras.layers.core.dense.Dense object at 0x2aab69a0aef0>]
compiled model


In [39]:
#model.fit(x, y, epochs=3)
model.fit(x, y)
#model.fit(x, y, epochs=10, shuffle=True)
# evaluate the model on the test set
test_y = test.drop(features.columns, axis=1)
test_x = test.drop(labels.columns, axis=1)
#test_x = test_x.drop("timedelta", axis=1)

loss, cat_accuracy, accuracy, mae = model.evaluate(test_x, test_y)
print("the accuracy on the training set is cat acc {}, reg acc {} and the mae is {}".format(cat_accuracy, accuracy, mae))

3218/3218 [==============================] - 137s 42ms/step - loss: 6.8932 - categorical_accuracy: 0.4645 - accuracy: 0.4645 - mae: 4.0265e-04
the accuracy on the training set is cat acc 0.4644761383533478, reg acc 0.4644761383533478 and the mae is 0.0004026502138003707


In [40]:
import numpy as np

#print(labels.columns)
# some pandas alchemy to sample  2 rows of each conversation
sample_df = df.sample(frac=1).reset_index(drop=True).groupby('conversation_id').apply(lambda x: x.sample(n=1)).reset_index(drop = True)
sample_features = take_features(sample_df, ["author", "current_time", "beam_node_time"])
sample_prediction = model.predict(sample_features)
np.matrix(sample_prediction)[0:5, -1] # the last row is the "new author column" label and should contain a high value

matrix([[0.5673113 ],
        [0.60965544],
        [0.5660656 ],
        [0.60802406],
        [0.5701584 ]], dtype=float32)

#### Predicting the author presence based on prediction probabilities
- compute predictions for the whole dataframe
- drop features and non-features except conversation and platform
- wide to long the authors to make them a index
- groupby conversation and platform

In [41]:
all_features = take_features(df, ["author", "current_time", "beam_node_time"])
predictions = model.predict(all_features)
column_names = labels.columns
predictions = pd.DataFrame(predictions, columns=column_names)
print(type(predictions))
print(predictions.shape)

<class 'pandas.core.frame.DataFrame'>
(514793, 3653)


In [42]:
all_non_features = df[["conversation_id", "platform"]]
print(type(all_non_features))
print(all_non_features.shape)
all_non_features.reset_index(drop=True, inplace=True)
joined_dataframe = all_non_features.join(predictions)
# not_needed_list = ["beam_node", "has_followed_path", "has_follow_path", "beam_node_author", "current"]
# author_predictions = joined_dataframe.drop(not_needed_list, axis=1)
# joined_dataframe.groupby(["platform", "conversation_id"]).mean()
# joined_dataframe["id"] = joined_dataframe.index

<class 'pandas.core.frame.DataFrame'>
(514793, 2)


In [43]:
joined_dataframe.Author_is_new.describe() # no idea why that is the same prediction of all the rows

count    5.147930e+05
mean     5.560513e-01
std      5.971310e-02
min      5.119365e-16
25%      5.469702e-01
50%      5.757409e-01
75%      5.847765e-01
max      9.999920e-01
Name: Author_is_new, dtype: float64

In [44]:
# joined_dataframe.describe()
joined_dataframe = joined_dataframe.groupby(["platform", "conversation_id"]).mean()
joined_dataframe.head(1)

,,Author_2700,Author_80313,Author_100895,Author_139565,Author_142486,Author_186373,Author_208252,Author_231696,Author_237258,Author_270213,...,Author_99775268,Author_99828634,Author_99859100,Author_99883498,Author_99922107,Author_99966650,Author_99974100,Author_99979114,Author_99983170,Author_is_new
platform,conversation_id,,,,,,,,,,,,,,,,,,,,,
reddit,174503,0.000001,0.000035,0.000004,0.000436,0.000008,0.000004,0.000001,0.000001,0.00001,0.000006,...,0.000023,0.000002,0.000001,0.000007,0.000063,0.000001,0.000004,0.000002,0.00001,0.590909


In [45]:


author_predictions_existing = joined_dataframe.drop(["Author_is_new"], axis=1)
author_predictions_existing.reset_index(level=['platform', 'conversation_id'],inplace=True)
author_predictions_existing_reshaped = pd.wide_to_long(author_predictions_existing, stubnames="Author_", i=['platform', 'conversation_id'], j="author_id")
author_predictions_existing_reshaped.head(3)


Author_
platform conversation_id author_id          
reddit   174503          2700       0.000001
                         80313      0.000035
                         100895     0.000004

In [46]:
# avg_author_pred = author_predictions_existing_reshaped.groupby(["platform", "conversation_id", "author_id"]).mean()
# avg_author_pred.head(3)

In [47]:
avg_conversation_pred  = author_predictions_existing_reshaped.groupby(["platform", "conversation_id"]).mean()
avg_conversation_pred.head(3)


Author_
platform conversation_id          
reddit   174503           0.000112
         203904           0.000115
         209098           0.000119

In [48]:
avg_platform_pred = avg_conversation_pred.groupby(["platform"]).mean()
print(avg_platform_pred)
avg_platform_pred # picking the correct author seems to be exceedingly difficult#


          Author_
platform         
reddit    0.00012


,Author_
platform,
reddit,0.00012


#### Notes
- inserting the new author column increased precision times 10
- categorical accuracy and regular accuracy match (which is weird)